In [16]:
import pandas as pd

# CSVファイルからデータを読み込む
df = pd.read_csv('japan_2022.csv')

In [17]:
# 欠損値の確認
print(df.isnull().sum())

# 欠損値の補完（例えば、前日の値で補完する）
# 'ffill'は前の値で欠損値を埋める方法
df.fillna(method='ffill', inplace=True)

# 移動平均の計算 (5日, 10日, 20日)
# 移動平均を特徴量として追加
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_20'] = df['Close'].rolling(window=20).mean()

# ボラティリティ（標準偏差の計算）
# ボラティリティを特徴量として追加
df['Volatility'] = df['Close'].rolling(window=5).std()

# 翌日の終値を目的変数に設定
df['Next_Close'] = df['Close'].shift(-1)

# 最後の行は目的変数がないため削除
df.dropna(inplace=True)

# データの確認
print(df.head())


Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
                   Date         Open         High          Low        Close  \
19  2022-02-01 00:00:00  27167.14063  27410.78906  27016.71094  27078.48047   
20  2022-02-02 00:00:00  27302.99023  27564.61914  27289.16016  27533.59961   
21  2022-02-03 00:00:00  27330.96094  27357.33008  27165.92969  27241.31055   
22  2022-02-04 00:00:00  27095.90039  27455.98047  27075.99023  27439.99023   
23  2022-02-07 00:00:00  27327.63086  27369.67969  27085.32031  27248.86914   

      Adj Close    Volume          MA_5         MA_10         MA_20  \
19  27078.48047  81100000  26795.886328  27146.156055  27862.712989   
20  27533.59961  85200000  26900.340234  27152.792969  27774.303517   
21  27241.31055  81100000  27114.542188  27099.631055  27669.761036   
22  27439.99023  79600000  27259.072266  27091.404101  27617.367090   
23  27248.86914  77100000  27308.450000  27057.45410

/var/folders/qv/99_g4bfn2t79tdbjt87pmqfh0000gn/T/ipykernel_9086/2321933690.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 説明変数と目的変数の設定
# 説明変数（特徴量）の選択
X = df[['Open', 'High', 'Low', 'Close', 'Volume', 'MA_5', 'MA_10', 'MA_20', 'Volatility']]
# 目的変数（ターゲット）の選択
y = df['Next_Close']

# データの分割
# 訓練データとテストデータに分割（80%訓練データ、20%テストデータ）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレスト回帰モデルの構築
model = RandomForestRegressor(n_estimators=100, random_state=42)
# モデルを訓練データで学習
model.fit(X_train, y_train)

# テストデータを用いて予測
y_pred = model.predict(X_test)

# 評価
# 平均二乗誤差（MSE）の計算
mse = mean_squared_error(y_test, y_pred)
# 平均絶対誤差（MAE）の計算
mae = mean_absolute_error(y_test, y_pred)

# 評価結果を表示
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

Mean Squared Error: 72716.95929381618
Mean Absolute Error: 205.2725493955608
